## Шаг 1. Изучение исходных данных

In [ ]:
# Загружаем все необходимые библиотеки
from scipy import stats as st
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import percentile
sns.set(color_codes=True)


In [ ]:
#Загружаем таблицы и передаем информации в переменные
data_calls = pd.read_csv('/datasets/calls.csv')
data_internet = pd.read_csv('/datasets/internet.csv')
data_messages = pd.read_csv('/datasets/messages.csv')
data_tariffs = pd.read_csv('/datasets/tariffs.csv')
data_users = pd.read_csv('/datasets/users.csv')



In [ ]:
data_calls.head(10)

In [ ]:
data_calls.info()

In [ ]:
data_calls['duration'].describe()

#### Краткий вывод по таблице звонков:
- call_date - столбец с информацией о дате имеет тип object, необходимо преобразовать в правильный тип datetime
- duration - есть нулевые значения. Необходимо проанализировать сколько их, откуда они, и как их отсеить. 
- пропусков нет, дубликаты определим на слудующем этапе анализа.
---

In [ ]:
data_internet.head(10)

In [ ]:
data_internet.info()

In [ ]:
data_internet['mb_used'].describe()

#### Краткий вывод по таблице трафика:
- session_date - столбец с информацией о дате имеет тип object, необходимо преобразовать в правильный тип datetime
- mb_used - есть нулевые значения. Необходимо проанализировать сколько их, откуда они, и как их отсеить 
- пропусков нет, дубликаты определим на слудующем этапе анализа.
---

In [ ]:
data_messages.head(10)

In [ ]:
data_messages.info()

#### Краткий вывод  по таблице сообщений:
- message_date - столбец с информацией о дате имеет тип object, необходимо преобразовать в правильный тип datetime
- пропусков нет, дубликаты определим на слудующем этапе анализа.
---

In [ ]:
data_tariffs.head(10)

In [ ]:
data_tariffs.info()

#### Краткий вывод  по таблице тарифов:
По этой таблице вопросов не возникает

In [ ]:
data_users.head(10)

In [ ]:
data_users.info()

#### Краткий вывод  по таблице пользователей:
- в столбце churn_date есть много пропусков согласно исходному описанию. Пропуски можно заменить индикаторным значением, чтобы их было видно на графиках.
- тип столбца reg_date необходимо преобразовать в datetime

###   Вывод по результатам предварительного изучения данных:
- во всех таблиц с датой call_date  , message_date , sessions_date , reg_date - столбец с информацией о дате имеет тип object, необходимо преобразовать в правильный тип datetime
duration - есть нулевые значения. Необходимо проанализировать сколько их, откуда они, и что с ними делать.
Необговоренных пропусков нет, дубликаты определим на слудующем этапе анализа.
----

 ##  Шаг 2. Предобработка данных

### Поиск дубликатов

In [ ]:
#  Проверим дубликаты во всех таблицах 
print('Дубликаты в таблице звонков: {:>8}'.format( data_calls.duplicated().sum()))
print('Дубликаты в таблице трафика: {:>8}'. format(data_internet.duplicated().sum()))
print('Дубликаты в таблице сообщений: {:>6}'. format(data_messages.duplicated().sum()))
print('Дубликаты в таблице пользователей: {:>2}'. format(data_users.duplicated().sum()))


### Преобразование даты

In [ ]:
# Проеобразуем столбец с датами в правильный тип данных
data_calls['call_date'] = pd.to_datetime(data_calls['call_date'], format ='%Y-%m-%d')
data_internet['session_date'] = pd.to_datetime(data_internet['session_date'], format ='%Y-%m-%d')
data_messages['message_date'] = pd.to_datetime(data_messages['message_date'], format ='%Y-%m-%d')
data_users['reg_date'] = pd.to_datetime(data_users['reg_date'], format ='%Y-%m-%d')


In [ ]:
#Проверяем результаты замены 
data_calls.info()

In [ ]:
#Проверяем результаты замены 
data_internet.info()

In [ ]:
#Проверяем результаты замены 
data_messages.info()

In [ ]:
#Проверяем результаты замены 
data_users.info()

### Анализ и отсеивание выбросов значений в столбце звонков "duration"

In [ ]:
# Проанализируем количество нулевых значений звонков
print('Количество нулевых звонков: {: >14}'.format(data_calls['duration'][data_calls['duration'] == 0 ].count()))
print('Максимальная длительность: {: >14}'.format(data_calls['duration'].max()))
print('Минимальная ненулевая длительность: {: >5}'.format(data_calls['duration'][data_calls['duration'] != 0 ].min()))

In [ ]:
# Распределение значений звонков
sns.distplot(data_calls['duration'], kde=True, rug=True)

In [ ]:
# Рассмотрим значения на графике с усами
sns.boxplot(y = data_calls['duration'] , palette='Blues')

In [ ]:
# По нижней границе исключаем значения меньше 25% квартиля. данные записываем в массив
q25 = percentile(data_calls['duration'] , 25)
data_calls = data_calls.loc[data_calls['duration'] >q25]
data_calls

In [ ]:
# Функция определяет выбросы и выводит обработанную таблицу
def outliers_clear(data, column , k):
    q25 , q75 = percentile(data[column] , 25) , percentile(data[column] , 75)
    # Граница выбросов сверху
    upper = q75 + (q75 - q25) * k
    return data[data[column] < upper]

# Функция печати количества выбросов и чистых данных
def outliers_print(data ,column , k):
    print('Количество выбросов при k-факторе {}: {:>16}'. format(k ,len(data[column]) - len(outliers_clear(data, column , k))))
    print('Количество принимаемых значений при k-факторе {}: {}'. format(k ,len(outliers_clear(data ,column , k))))

In [ ]:
# Проверим распределение значений с отсеиванием выбросов с коэффициентом 1,5
outliers_print(data_calls , 'duration' , 1.5)
data_calls_clear_1 = outliers_clear(data_calls , 'duration' , 1.5)
plt.figure(figsize = (10,5))
plt.title('График распределения при k=1.5')
sns.distplot(data_calls_clear_1['duration'], kde=True, rug=True)
plt.show()


In [ ]:
# Проверим распределение значений с отсеиванием выбросов с коэффициентом 2
outliers_print(data_calls , 'duration' , 2)
data_calls_clear_2 = outliers_clear(data_calls , 'duration' , 2)
plt.figure(figsize = (10,5))
plt.title('График распределения при k=2')
sns.distplot(data_calls_clear_2['duration'], kde=True, rug=True)
plt.show()

In [ ]:
# Boxplot звонков без выбросов
plt.figure(figsize=(5,7))
sns.boxplot(y = data_calls_clear_1['duration'] , palette='Blues')
plt.show()

In [ ]:
# Записываем выбранную таблицу в новый датафрейм
data_calls_clear = data_calls_clear_1

### Анализ и отсеивание выбросов значений в столбце трафика "mb_used"

In [ ]:
# Проанализируем количество нулевых значений трафика
print('Количество сессий с нулевым значением трафика: {: >5}'.format(data_internet['mb_used'][data_internet['mb_used'] == 0 ].count()))
print('Максимальная длительность сессии: {: >20}'.format(data_internet['mb_used'].max()))
print('Минимальная ненулевая длительность сессии: {: >8}'.format(data_internet['mb_used'][data_internet['mb_used'] != 0 ].min()))


In [ ]:
# Распределение значений трафика
sns.distplot(data_internet['mb_used'], kde=True, rug=True, color = 'green' )

In [ ]:
# Рассмотрим значения на графике с усами
sns.boxplot(y = data_internet['mb_used'] , palette='Greens')

In [ ]:
# По нижней границе исключаем значения меньше 50МБ. 
data_internet = data_internet.loc[data_internet['mb_used'] >50]
data_calls

In [ ]:
# Проверим распределение значений с отсеиванием выбросов с коэффициентом 1.5
outliers_print(data_internet , 'mb_used' , 1.5)
data_internet_clear_1= outliers_clear(data_internet , 'mb_used' , 1.5)
plt.figure(figsize = (10,5))
plt.title('График распределения при k=1.5')
sns.distplot(data_internet_clear_1['mb_used'], kde=True, rug=True , color = 'green')
plt.show()

In [ ]:
# Проверим распределение значений с отсеиванием выбросов с коэффициентом 2
outliers_print(data_internet , 'mb_used' ,2)
data_internet_clear_2= outliers_clear(data_internet , 'mb_used' , 2)
plt.figure(figsize = (10,5))
plt.title('График распределения при k=2')
sns.distplot(data_internet_clear_2['mb_used'], kde=True, rug=True , color = 'green')
plt.show()

In [ ]:
# Boxplot трафика без выбросов
plt.figure(figsize=(5,7))
sns.boxplot(y = data_internet_clear_1['mb_used'] , palette='Greens')
plt.show()

In [ ]:
data_internet_clear = data_internet_clear_1

### Расчет количества сделанных звонков и израсходованных минут разговора по месяцам

### Расчет количества отправленных сообщений по месяцам

### Расчет объема израсходованного интернет-трафика по месяцам

### Расчет помесячной выручки с каждого пользователя

###   Вывод по результатам предобработки данных:


----

 ##  Шаг 3. Анализ данных

###   Вывод по результатам предварительного изучения данных:
----

##  Шаг 4. Проверка гипотез

In [ ]:
# identify outliers with interquartile range
from numpy.random import seed
from numpy.random import randn
from numpy import percentile

# seed the random number generator
seed(1)

# generate univariate observations
data = 5 * randn(10000) + 50

# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off

# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))

# remove outliers
outliers_removed = [x for x in data if x > lower and x < upper]
print('Non-outlier observations: %d' % len(outliers_removed))
print(q75*1.5)

###   Вывод по результатам проверки гипотез:
----

##  Шаг 5. Общий вывод